In [2]:
import librosa  
import numpy as np  

In [3]:
def extract_features(audio_path):
    data, sr = librosa.load(audio_path, res_type='kaiser_fast')
    mfccs = np.mean(librosa.feature.mfcc(y=data, sr=sr, n_mfcc=128).T, axis=0)
    mfccs = np.mean(mfccs.T, axis=0)
    return mfccs

In [ ]:
X = []
Y = []